In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [2]:
# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'
def download(filename, expected_bytes):
    if not os.path.exists(filename):
        filename,_ = urllib.request.urlretrieve(url+filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
        
    return filename 
filename = download('text8.zip', 31344016)

Found and verified text8.zip


In [4]:
# Read the data into a list of strings.
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data=tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

vocabulary = read_data(filename)
print('Data size', len(vocabulary))

Data size 17005207


In [9]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000

def build_dataset(words, n_words):
    count=[['UNK',-1]]
    count.extend(collections.Counter(words).most_common(n_words-1))
    dictionary = dict()
    for word,_ in count:
        dictionary[word] = len(dictionary)
    data=list()
    unk_count=0
    for word in words:
        if word in dictionary:
            index=dictionary[word]
        else:
            index=0
            unk_count+=1
        data.append(index)
    count[0][1]=unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data,count,dictionary,reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)

In [12]:
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5240, 3081, 12, 6, 195, 2, 3136, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [17]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size%num_skips==0
    assert num_skips<=2*skip_window
    batch=np.ndarray(shape=(batch_size), dtype=np.int32)
    labels=np.ndarray(shape=(batch_size,1), dtype=np.int32)
    span = 2*skip_window+1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index+span > len(data):
        data_index=0
    buffer.extend(data[data_index:data_index+span])
    data_index+=span
    
    for i in range(batch_size//num_skips):
        target = skip_window # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0,span-1)
                
            targets_to_avoid.append(target)
            batch[i*num_skips+j] =buffer[skip_window]
            labels[i*num_skips+j] = buffer[target]
        if data_index==len(data):
            buffer[:] = data[:span]
            data_index=span
        else:
            buffer.append(data[data_index])
            data_index+=1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index+len(data)-span)%len(data)
    return batch,labels
            

In [18]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i],reverse_dictionary[batch[i]],'->',labels[i,0], reverse_dictionary[labels[i,0]])

3081 originated -> 12 as
3081 originated -> 5240 anarchism
12 as -> 3081 originated
12 as -> 6 a
6 a -> 195 term
6 a -> 12 as
195 term -> 2 of
195 term -> 6 a


In [21]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph=tf.Graph()

with graph.as_default():
    
    # input data
    train_inputs = tf.placeholder(tf.int32,shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # Ops and variables pinned to the CPU because of missing GPU implementation
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size],-1.0,1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],stddev = 1.0/math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros(vocabulary_size))
    
    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    
    loss = tf.reduce_mean(tf.nn.nce_loss(biases=nce_biases, weights= nce_weights, labels=train_labels, inputs=embed,
                                        num_sampled=num_sampled, num_classes=vocabulary_size))
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    
    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings/norm 
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    # Add variable initializer.
    init = tf.global_variables_initializer()
    
    # Step 5: Begin training.
    num_steps = 100001

    with tf.Session(graph=graph) as session:
        init.run()
        print('initialized')
        
        average_loss=0
        for step in xrange(num_steps):
            batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
            
            _, loss_val = session.run([optimizer,loss], feed_dict=feed_dict)
            average_loss += loss_val
            
            if step%2000 ==0:
                if step>0:
                    average_loss/=2000
                print('average loss at step', step, ':', average_loss)
                average_loss=0
            
            if step%10000==0:
                sim = similarity.eval()
                for i in xrange(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k =8
                    nearest = (-sim[i,:]).argsort()[1:top_k+1]
                    log_str = 'Nearest to %s: ' %valid_word
                    for k in range(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s' %(log_str, close_word)
                    print(log_str)
                    
        final_embeddings = normalized_embeddings.eval()
        
        

initialized
average loss at step 0 : 264.097106934
Nearest to up:  krause java bigcup recompilation nther descendants harriers headers
Nearest to some:  sergio primary neva byzantine rahul stove nde scotch
Nearest to and:  tramp launceston stride vitamin lindbergh processed wreck detergents
Nearest to from:  karaoke unites contend flavor xxii peloponnesus kh tapes
Nearest to people:  redaction gt gordie delayed codepages denton environmentalist phil
Nearest to three:  largely inen railroads dahmer lancashire theatrical cds nord
Nearest to during:  pills stow navel bonner kill yi coerce rosary
Nearest to s:  waging blunder defying caters homeschool mainstream harriman volleyball
Nearest to will:  delano partly iconoclastic dingo ratifying ordering putnam greenlandic
Nearest to d:  integrins heredity warfield zarah weighing dreadful decidable panther
Nearest to he:  dubrovnik waiver millsaps icmp replenished chopper purchased beattie
Nearest to into:  dreamland dreary relax interchanges 

In [ ]:
# Step 6: Visualize the embeddings.